In [1]:
from cryptography.hazmat.primitives.ciphers.algorithms import AES
from cryptography.hazmat.primitives.ciphers import modes, Cipher
from cryptography.hazmat.backends import default_backend
from os import urandom
import sys
from math import ceil

In [2]:
ENDIAN = "big"

def xor(x, y):
    assert len(x) == len(y)
    a = int.from_bytes(x, ENDIAN)
    b = int.from_bytes(y, ENDIAN)
    r = a ^ b
    return r.to_bytes(len(x), ENDIAN)


def ctr_result(k, iv, x, blocksize, ivsize):

    cipher = Cipher(AES(k), modes.ECB(), backend=default_backend())
    aes = cipher.encryptor().update
    
    long_k = bytearray()
    n = ceil(len(x) / blocksize)
    
    for i in range(0, n):
        c = aes((iv+i).to_bytes(ivsize, ENDIAN))
        long_k += c
    
    return xor(x, long_k[:len(x)])


def encrypt_ctr(pt_string, key_string, blocksize=16, ivsize=16):
    
    k = bytes.fromhex(key_string)
    pt = bytes(pt_string, 'utf-8')
    
    iv_raw = urandom(ivsize)
    iv = int.from_bytes(iv_raw, ENDIAN)
    
    return iv_raw.hex() + ctr_result(k, iv, pt, blocksize, ivsize).hex()
    
    
def decrypt_ctr(ct_string, key_string, blocksize=16, ivsize=16):

    k = bytes.fromhex(key_string)
    c = bytes.fromhex(ct_string)
    iv = int.from_bytes(c[:ivsize], ENDIAN)
    return str(ctr_result(k, iv, c[ivsize:], blocksize, ivsize), 'utf-8')

In [3]:
key_string = "36f18357be4dbd77f050515c73fcf9f2"
ivct_string1 ="69dda8455c7dd4254bf353b773304eec0ec7702330098ce7f7520d1cbbb20fc388d1b0adb5054dbd7370849dbf0b88d393f252e764f1f5f7ad97ef79d59ce29f5f51eeca32eabedd9afa9329"
ivct_string2 = "770b80259ec33beb2561358a9f2dc617e46218c0a53cbeca695ae45faa8952aa0e311bde9d4e01726d3184c34451"

m_string1 = "CTR mode lets you build a stream cipher from a block cipher."
m_string2 = "Always avoid the two time pad!"

print(decrypt_ctr(ivct_string1, key_string))
print(decrypt_ctr(ivct_string2, key_string))

c_string1 = encrypt_ctr(m_string1, key_string)
c_string2  = encrypt_ctr(m_string2, key_string)

print(decrypt_ctr(c_string1, key_string))
print(decrypt_ctr(c_string2, key_string))

assert decrypt_ctr(c_string1, key_string) == decrypt_ctr(ivct_string1, key_string)
assert decrypt_ctr(c_string2, key_string) == decrypt_ctr(ivct_string2, key_string)
assert decrypt_ctr(ivct_string1, key_string) == m_string1
assert decrypt_ctr(ivct_string2, key_string) == m_string2

CTR mode lets you build a stream cipher from a block cipher.
Always avoid the two time pad!
CTR mode lets you build a stream cipher from a block cipher.
Always avoid the two time pad!
